<h1>Sentiment Polarity</h1>
<p>In questo notebook andremo ad utilizzare la libreria SentiWordNet per calcolare il sentiment contenuto in varie parole contenute nelle recensioni.

In [1]:
import os

import sys
sys.path.append("..")

import numpy as np
import pandas as pd
import textblob as tb

from nltk import pos_tag
from nltk.corpus import sentiwordnet as swn

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

from scripts.plotting_functions import plot_words_polarity
from scripts.utility import convert_tags, get_pos_score
from scripts.analyzers import remove_stopwords, dummy_analyzer

from bokeh.plotting import output_notebook, show

output_notebook()

DATA_FOLDER = os.path.join(os.getcwd(), "../data")

Loading BokehJS ...

<h3>Caricamento Dati e Sampling</h3>
<p>Cominciamo con il leggere il file di training contenente le recensioni e caricarlo in un <code>DataFrame</code> pandas. Successivamente si è rilevato necessario effettuare sampling per ridurre in numero di recensioni considerate; questo perchè la funzione <code>TextBlob()</code>, utilizzata per il <i>tagging</i> delle parole, risulta particolarmente lenta.

In [2]:
train_fp = os.path.join(DATA_FOLDER, "csv/train.csv")
reviews = pd.read_csv(train_fp, index_col="review_id", usecols=['text', 'stars', 'review_id'])
reviews = reviews.sample(10000)

<p>Procediamo con la rimozione delle <i>stopwords</i> dalle recensioni.

In [3]:
reviews['text'] = reviews['text'].apply(remove_stopwords)

<p>Calcoliamo la <i>part-of-speach</i> a cui sono associate le varie parole contenute nelle recensioni. Per farlo abbiamo sfruttato la libreria <code>textblob</code> che espone la classe <code>TextBlob(word)</code>. Quest'ultima permette di costuire un'istanza della classe sulla base di un testo. Questa classe espone svariati metodi e attributi, di particolare interesse per i nostri scopi l'attributo <code>.tags</code>. Quest'ultimo contiene una lista di tuple, una per ogni parola della recensione, i cui elementi sono rispettivamente una parola della recensione e la relativa <i>part-of-speach</i>.

In [4]:
reviews['pos'] = reviews['text'].apply(lambda x : tb.TextBlob(x).tags)

<p>Sfortunatamente la sintassi utilizzata da <code>TextBlob</code> per identificare le tags che compaiono nel testo è leggermente differente dalle tags utilizzate da <code>SentiWordNet</code>, è dunque necessaria una traduzione.<br>
Per questa operazione abbiamo sviluppato manualmente un dizionario che mappi le tags di <code>TextBlob</code> nelle tags di <code>SentiWordNet</code>.

In [5]:
reviews['pos'] = reviews['pos'].apply(convert_tags)

<p>Dal momento che, oltre alla polarità delle parole, siamo anche interessati alla loro frequenza è necessario che esssa venga calcolata. Per farlo abbiamo costruito, tramite la classe <code>CountVectorizer</code> una <i>term-matrix</i> che per ogni recensione ed ogni parola nel corpus ne indica la relativa frequenza. Successivamente sommando i valori lungo le colonne abbiamo ottenuto una lista in cui, ad ogni parola, è associata la frequenza assoluta nel corpus di recensioni.

In [6]:
cv = CountVectorizer(analyzer=dummy_analyzer)
bow = cv.fit_transform(reviews['pos'])

In [7]:
word_freq = pd.DataFrame(columns=['word', 'freq'])

In [8]:
freqs = bow.sum(axis=0)

In [9]:
for k in cv.vocabulary_:
    tmp_df = pd.DataFrame({'word':[k], 'freq':[freqs[0, cv.vocabulary_[k]-1]]})
    word_freq = word_freq.append(tmp_df)

<p>A questo punto possiamo utilizzare le funzioni <code>.pos_score()</code> e <code>.neg_score()</code> della libreria <code>SentiWordNet</code>. Queste funzioni ritornano, sulla base di una parola ed una tag, una misura della positività o negatività espressa dalla parola stessa. Per rendere il codice più leggibile tali funzioni sono state definite in un file a parte nella cartella <code>scripts</code>.

In [10]:
word_freq['pos_score'] = word_freq['word'].apply(lambda x : get_pos_score(x, True))
word_freq['neg_score'] = word_freq['word'].apply(lambda x : get_pos_score(x, False))

<p>Per molte combinazioni di parole e tags <code>SentiWordNet</code> non è in grado di fornire un coefficiente di positività o negatività. Le combinazioni per cui non è stato possibile calcolare il sentiment hanno nelle relative celle il valore speciale <code>nan</code>. Provvediamo a eliminare tutte le righe che contengano uno o più valori <code>nan</code>.

In [11]:
word_freq = word_freq.dropna(axis=0, how='any')

<p>Per altre parole i coefficienti risultano essere uguali a zero, selezioniamo dunque tutte le righe del DataFrame per cui almeno uno dei due campi sia diverso da zero.

In [12]:
word_freq = word_freq[(word_freq['pos_score'] != 0) | (word_freq['neg_score'] != 0)]

<p>Ordiniamo le parole in funzione della frequenza, del coefficiente di sentimento positivo e negativo.

In [13]:
word_freq = word_freq.sort_values(by=['freq', 'pos_score', 'neg_score'], ascending=False)

<p>Per poter mettere a confronto i coefficienti di sentiment positivi e negativi è più comodo considerare il valore assoluto di quest'ultimi. Applichiamo dunque alla colonna <code>'neg_score'</code> una funzione che, per ogni valore, ne ritorni il valore assoluto.

In [14]:
word_freq['neg_score'] = word_freq['neg_score'].apply(lambda x : abs(x))

<p>Terminiamo con una rappresentazione grafica dei coefficienti di sentiment positivi e negativi per le parole più frequenti all'interno del corpus di recensioni.

In [15]:
n = 35
fig = plot_words_polarity(
    word_freq,
    n,
    title='Sentiment Polarity delle {} parole più frequenti nel sample di recensioni'.format(n),
    x_axis_label='Sentiment',
    y_axis_label='Parole più frequenti'
)

In [16]:
show(fig)